# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of force balance test
start = Time("2023-04-18 16:10:00Z", scale='utc')
end = Time("2023-04-18 16:15:00Z", scale='utc')

In [ ]:
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)

In [ ]:
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plot_counter = 0
for i in range(3):
    for j in range(2):
        smoothed_ims_data = ims_data[plots[plot_counter]].rolling(100).mean()
        smoothed_hardpoint_data = hardpoint_data[plots[plot_counter]].rolling(100).mean()
        if plot_counter > 2:
            smoothed_ims_data *= 3600.0
            smoothed_hardpoint_data *= 3600.0
            unit = 'arcsec'
        else:
            smoothed_ims_data *= 1000.0
            smoothed_hardpoint_data *= 1000.0
            unit = 'mm'
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'Hardpoint ({unit})')
        ax = axs[i][j].twinx()
        ax.set_ylabel(f'IMS ({unit})')
        smoothed_hardpoint_data.plot(ax=axs[i][j], color='blue', label='Hardpoint')
        smoothed_ims_data.plot(ax=ax, color='red', label='IMS')
        plot_counter += 1
timestamp = hardpoint_data.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Data_{timestamp}.png")